In [1]:
#The data, once extracted, is contained within 22 files with .sgm file extension, a legacy markup format.
#To proceed, we need to convert to .xml.  I will use the unix command osx from the library OpenSP.
#OpenSP is not a standard library, but can be installed via the Homebrew Package Manager.
#Homebrew is built for the macOS operating system.
#For other environments, a different solution will need to be found to obtain OpenSP.

#The installation of Homebrew was done with the following command at terminal:
#ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)" < /dev/null 2> /dev/null

#With Homebrew installed, I installed the OpenSP package with the following command at terminal:
#brew install open-sp

#Inspiration for the data representation and machine learning methods came from the following website
#and from the corresponding kaggle competition:
#https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff

import requests
import tempfile
from os import path
import tarfile
import subprocess
from bs4 import BeautifulSoup
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
import collections
from scipy.stats import t
from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, hamming_loss
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix

In [2]:
#Data acquisition

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/reuters21578-mld/reuters21578.tar.gz'
resp = requests.get(url)
data_dict = {'title':[], 'text':[], 'topic_set':[]}
corpus_topic_set = set()

with tempfile.TemporaryDirectory() as tmp_dir:
    targz_path = path.join(tmp_dir, 'reuters21578.tar.gz')
    with open(targz_path, 'wb') as targz_file:
        targz_file.write(resp.content)
    with tarfile.open(targz_path, 'r:gz') as targz_file:
        targz_file.extractall(path=tmp_dir)
    for num in range(22):
        sgm_path = path.join(tmp_dir, 'reut2-0' + str(num).zfill(2) + '.sgm')
        with open(sgm_path, 'r', encoding='cp1252') as sgm_file:
            xml_text = subprocess.run(['osx', '--directory=' + tmp_dir], stdin=sgm_file, \
                stdout=subprocess.PIPE, encoding='cp1252').stdout
        soup = BeautifulSoup(xml_text, 'xml')
        for article in soup.find_all('REUTERS'):
            if (article.TOPICS is not None) & (article.TOPICS.D is not None) & \
                (article.TITLE is not None) & (article.BODY is not None):
                topic_set = {re.sub('-', '', topic.string) for topic in article.TOPICS.find_all('D')}
                corpus_topic_set.update(topic_set)
                data_dict['topic_set'].append(topic_set)
                data_dict['title'].append(article.TITLE.string)
                data_dict['text'].append(article.BODY.string)
for topic in corpus_topic_set:
    data_dict[topic] = [int(topic in topic_set) for topic_set in data_dict['topic_set']]
del data_dict['topic_set']
data_df = pd.DataFrame(data_dict)
binary_df = data_df.drop(columns=['title', 'text'])
drop_columns = binary_df.columns[binary_df.sum(axis='index') <= 10].values
data_df.drop(columns=drop_columns, inplace=True)
data_df['topic_count'] = data_df.sum(axis='columns', numeric_only=True)
data_df = data_df[data_df.topic_count > 0]

KeyboardInterrupt: 

In [ ]:
data_df.head()

In [ ]:
data_df.info()

In [ ]:
#Exploratory Data Analysis using CountVectorizer without tuned parameters

eda_text_vectorizer = CountVectorizer(strip_accents='unicode', stop_words='english')
eda_text_matrix = eda_text_vectorizer.fit_transform(data_df.text)
eda_topic_matrix = data_df.drop(columns=['title', 'text', 'topic_count']).values

In [ ]:
plt.spy(eda_topic_matrix, markersize=0.5, aspect='auto')
plt.title('Visualization of Topic Matrix Data (Filled Cell Represents Nonzero entry)')
plt.xlabel('Distinct Topics')
plt.ylabel('Document')
print('Proportion of nonzero entries: ' + str(np.count_nonzero(eda_topic_matrix)/len(eda_topic_matrix.flatten())))

In [ ]:
plt.spy(eda_text_matrix, markersize=0.01, aspect='auto')
plt.title('Visualization of Text Matrix Data (Filled Cell Represents Nonzero entry)')
plt.xlabel('Distinct Words')
plt.ylabel('Document')
print('Proportion of nonzero entries: ' + str(eda_text_matrix.getnnz()/len(eda_text_matrix.toarray().flatten())))

In [ ]:
plt.hist(data_df.topic_count, log=True, bins=14)
plt.title('Topics count histogram (Log scale)')
plt.xlabel('Number of topics for an article')
plt.ylabel('Log count')

In [ ]:
eda_topic_occurence = data_df.drop(columns=['title', 'text', 'topic_count']).sum(axis='index').values
eda_topic_percentiles = np.percentile(eda_topic_occurence, [0, 50, 75, 95, 99, 99.9, 100])
sns.kdeplot(np.log10(eda_topic_occurence), shade=True)
plt.title('KDE estimation of Log Topic Occurence in Corpus Distribution')
plt.xlabel('Log Topic Occurence in Corpus')
plt.ylabel('Probability Density')
print('Minimum topic occurence: ' + str(eda_topic_percentiles[0]))
print('Median topic occurence: ' + str(eda_topic_percentiles[1]))
print('75th percentile topic occurence: ' + str(eda_topic_percentiles[2]))
print('95th percentile topic occurence: ' + str(eda_topic_percentiles[3]))
print('99th percentile topic occurence: ' + str(eda_topic_percentiles[4]))
print('99.9th percentile topic occurence: ' + str(eda_topic_percentiles[5]))
print('Maximum topic occurence: ' + str(eda_topic_percentiles[6]))

In [ ]:
eda_word_occurence = np.count_nonzero(eda_text_matrix.toarray(), axis=0)
eda_word_percentiles = np.percentile(eda_word_occurence, [0, 50, 75, 95, 99, 99.9, 100])
sns.kdeplot(np.log10(eda_word_occurence), shade=True)
plt.title('KDE estimation of Log Word Occurence in Corpus Cumulative Distribution')
plt.xlabel('Log Word Occurence in Corpus')
plt.ylabel('Probability Density')
print('Minimum word occurence: ' + str(eda_word_percentiles[0]))
print('Median word occurence: ' + str(eda_word_percentiles[1]))
print('75th percentile word occurence: ' + str(eda_word_percentiles[2]))
print('95th percentile word occurence: ' + str(eda_word_percentiles[3]))
print('99th percentile word occurence: ' + str(eda_word_percentiles[4]))
print('99.9th percentile word occurence: ' + str(eda_word_percentiles[5]))
print('Maximum word occurence: ' + str(eda_word_percentiles[6]))

In [ ]:
# Statistical Inference

single_topic_df = data_df[data_df.topic_count == 1].reset_index(drop=True).drop(columns='topic_count')
st_binary_df = single_topic_df.drop(columns=['title', 'text'])
st_drop_columns = st_binary_df.columns[st_binary_df.sum(axis='index') <= 10].values
single_topic_df.drop(columns=st_drop_columns, inplace=True)
single_topic_df.info()

In [ ]:
def centroid(matrix):
    m, n = matrix.shape
    cent = np.zeros(n)
    for i in range(n):
        cent[i] = matrix[:,i].mean()
    return cent
def square_distance(vector_1, vector_2):
    sq_dist = 0
    for i in range(len(vector_1)):
        sq_dist += (vector_2[i] - vector_1[i]) ** 2
    return sq_dist
def closest_distance(matrix):
    m, n = matrix.shape
    pairwise_distance = np.zeros((m,m))
    for i in range(m):
        for j in range(i+1,m):
            pairwise_distance[i,j] = square_distance(matrix[i], matrix[j])
    return np.unravel_index(np.argmax(pairwise_distance), pairwise_distance.shape)
def closest_centroids_inf(inf_data_df, inf_vectorizer):
    inf_vectorizer.fit(inf_data_df.text)
    topics = inf_data_df.drop(columns=['title','text']).columns.values
    inf_dict = {'topic':[], 'topic_matrix':[], 'topic_centroid':[]}
    for topic in topics:
        inf_dict['topic'].append(topic)
        mask = inf_data_df[topic].astype('bool')
        topic_matrix = inf_vectorizer.transform(inf_data_df[mask].text).toarray()
        inf_dict['topic_centroid'].append(centroid(topic_matrix))
        inf_dict['topic_matrix'].append(topic_matrix)
    inf_df = pd.DataFrame(inf_dict)
    centroids = np.concatenate(inf_df.topic_centroid.values).reshape((len(inf_df.index),\
        len(inf_df.loc[0,'topic_centroid'])))
    index_1, index_2 = closest_distance(centroids)
    topic_1 = inf_df.loc[index_1, 'topic']
    topic_2 = inf_df.loc[index_2, 'topic']
    topic_1_matrix = inf_df.loc[index_1, 'topic_matrix']
    topic_2_matrix = inf_df.loc[index_2, 'topic_matrix']
    topic_1_centroid = inf_df.loc[index_1, 'topic_centroid']
    topic_2_centroid = inf_df.loc[index_2, 'topic_centroid']
    topic_distance = np.sqrt(square_distance(topic_1_centroid, topic_2_centroid))
    topic_1_doc_count, _ = topic_1_matrix.shape
    topic_2_doc_count, _ = topic_2_matrix.shape
    topic_1_variance = 0
    topic_2_variance = 0
    for doc in range(topic_1_doc_count):
        topic_1_variance += square_distance(topic_1_matrix[doc], topic_1_centroid)
    for doc in range(topic_2_doc_count):
        topic_2_variance += square_distance(topic_2_matrix[doc], topic_2_centroid)
    topic_1_variance *= 1 / (topic_1_doc_count - 1)
    topic_2_variance *= 1 / (topic_2_doc_count - 1)
    degrees_of_freedom = topic_1_doc_count + topic_2_doc_count - 2
    standard_error = np.sqrt(((topic_1_doc_count - 1) * topic_1_variance + (topic_2_doc_count - 1) \
        * topic_2_variance) / degrees_of_freedom * (1 / topic_1_doc_count + 1 / topic_2_doc_count))
    t_stat = topic_distance / standard_error
    p_val = 1 - t.cdf(t_stat, degrees_of_freedom)
    return topic_1, topic_2, t_stat, p_val

In [ ]:
%%time
inf_count_vectorizer = CountVectorizer(strip_accents='unicode', stop_words='english')
topic_1_count, topic_2_count, t_stat_count, p_val_count = closest_centroids_inf(single_topic_df, inf_count_vectorizer)
print('The closest 2 topics in our count vector space were: ', topic_1_count, topic_2_count)
print('The t statistic for the difference of topic centroids in the count space was: ', t_stat_count, '\n', \
   ' with associated p value: ', p_val_count)

In [ ]:
%%time
inf_bin_vectorizer = CountVectorizer(strip_accents='unicode', stop_words='english', binary=True)
topic_1_bin, topic_2_bin, t_stat_bin, p_val_bin = closest_centroids_inf(single_topic_df, inf_bin_vectorizer)
print('The closest 2 topics in our binary vector space were: ', topic_1_bin, topic_2_bin)
print('The t statistic for the difference of topic centroids in the binary space was: ', t_stat_bin, '\n', \
   ' with associated p value: ', p_val_bin)

In [ ]:
%%time
inf_tfidf_vectorizer = TfidfVectorizer(strip_accents='unicode', stop_words='english')
topic_1_tfidf, topic_2_tfidf, t_stat_tfidf, p_val_tfidf = closest_centroids_inf(single_topic_df, inf_tfidf_vectorizer)
print('The closest 2 topics in our tfidf vector space were: ', topic_1_tfidf, topic_2_tfidf)
print('The t statistic for the difference of topic centroids in the tfidf space was: ', t_stat_tfidf, '\n', \
   ' with associated p value: ', p_val_tfidf)

In [ ]:
# Multilabel Classification

double_topic_df = data_df[data_df.topic_count == 2].reset_index(drop=True).drop(columns='topic_count')
dt_binary_df = double_topic_df.drop(columns=['text', 'title'])
st_common_topics = st_binary_df.sum(axis='index').sort_values(ascending=False).head(n=5).index.values
dt_common_topics = dt_binary_df.sum(axis='index').sort_values(ascending=False).head(n=5).index.values
common_topics = np.union1d(st_common_topics, dt_common_topics)
simple_df = data_df[np.append(np.array(['title', 'text']), common_topics)]
simple_df = simple_df[simple_df.sum(axis='columns', numeric_only=True) > 0]

smpl_train_df, smpl_test_df = train_test_split(simple_df, random_state=1)
smpl_clf_text_vectorizer = TfidfVectorizer(strip_accents='unicode', stop_words='english')
smpl_clf_text_vectorizer.fit(simple_df.text)
smpl_x_train = smpl_clf_text_vectorizer.transform(smpl_train_df.text)
smpl_y_train = smpl_train_df.drop(columns=['title', 'text'])
smpl_x_test = smpl_clf_text_vectorizer.transform(smpl_test_df.text)
smpl_y_test = smpl_test_df.drop(columns=['title', 'text'])

train_df, test_df = train_test_split(data_df, random_state=1)
clf_text_vectorizer = TfidfVectorizer(strip_accents='unicode', stop_words='english')
clf_text_vectorizer.fit(data_df.text)
x_train = clf_text_vectorizer.transform(train_df.text)
y_train = train_df.drop(columns=['title', 'text'])
x_test = clf_text_vectorizer.transform(test_df.text)
y_test = test_df.drop(columns=['title', 'text'])

In [ ]:
%%time
print('Binary Relevance with Gaussian Naive Bayes (simplified dataset)', '\n')
brgnb_smpl_classifier = BinaryRelevance(GaussianNB())
brgnb_smpl_classifier.fit(smpl_x_train, smpl_y_train)
brgnb_smpl_y_pred = brgnb_smpl_classifier.predict(smpl_x_test)
print('Test Accuracy: ', accuracy_score(smpl_y_test, brgnb_smpl_y_pred))
print('Test Hamming loss: ', hamming_loss(smpl_y_test, brgnb_smpl_y_pred))

In [ ]:
%%time
print('One vs Rest classifier with Logistic Regression (simplified dataset)', '\n')
logreg_smpl_pipeline = Pipeline([('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),])
for topic in common_topics:
    logreg_smpl_pipeline.fit(smpl_x_train, smpl_y_train[topic])
    lrp_smpl_y_pred = logreg_smpl_pipeline.predict(x_test)
    print('Test Accuracy for topic ' + topic + ': ' + str(accuracy_score(smpl_y_test[topic], lrp_smpl_y_pred)))

In [ ]:
%%time
print('Classifier Chain with Logistic Regression (simplified dataset)', '\n')
cclr_smpl_classifier = ClassifierChain(LogisticRegression())
cclr_smpl_classifier.fit(smpl_x_train, smpl_y_train)
cclr_smpl_y_pred = cclr_smpl_classifier.predict(smpl_x_test)
print('Test Accuracy: ' + str(accuracy_score(smpl_y_test, cclr_smpl_y_pred)))
print('Test Hamming Loss: ' + str(hamming_loss(smpl_y_test, cclr_smpl_y_pred)))

In [ ]:
%%time
print('Label Powerset with Logistic Regression (simplified dataset)', '\n')
lplr_smpl_classifier = LabelPowerset(LogisticRegression(solver='sag'))
lplr_smpl_classifier.fit(smpl_x_train, smpl_y_train)
lplr_smpl_y_pred = lplr_smpl_classifier.predict(smpl_x_test)
print('Test Accuracy: ' + str(accuracy_score(smpl_y_test, lplr_smpl_y_pred)))
print('Test Hamming Loss: ' + str(hamming_loss(smpl_y_test, lplr_smpl_y_pred)))

In [ ]:
%%time
print('Binary Relevance with Logistic Regression (simplified dataset)', '\n')
brlr_smpl_classifier = BinaryRelevance(LogisticRegression())
brlr_smpl_classifier.fit(smpl_x_train, smpl_y_train)
brlr_smpl_y_pred = brlr_smpl_classifier.predict(smpl_x_test)
print('Test Accuracy: ', accuracy_score(smpl_y_test, brlr_smpl_y_pred))
print('Test Hamming loss: ', hamming_loss(smpl_y_test, brlr_smpl_y_pred))

In [ ]:
#print('Multi-label k-Nearest Neighbors', '\n')
#x_train_dense = lil_matrix(x_train).toarray()
#y_train_dense = lil_matrix(y_train).toarray()
#x_test_dense = lil_matrix(x_test).toarray()
#mlknn_classifier = MLkNN(k=10)
#mlknn_classifier.fit(x_train_dense, y_train_dense)
#mlknn_y_pred = mlknn_classifer.predict(x_test_dense)
#print('Test Accuracy: ' + str(accuracy_score(y_test, mlknn_y_pred)))
#print('Test Hamming Loss: ' + str(hamming_loss(y_test, mlknn_y_pred)))

In [ ]:
#%%time
#print('Binary Relevance with Gaussian Naive Bayes', '\n')
#brgnb_classifier = BinaryRelevance(GaussianNB())
#brgnb_classifier.fit(x_train, y_train)
#brgnb_y_pred = brgnb_classifier.predict(x_test)
#print('Test Accuracy: ', accuracy_score(y_test, brgnb_y_pred))
#print('Test Hamming loss: ', hamming_loss(y_test, brgnb_y_pred))

In [ ]:
%%time
print('One vs Rest classifier with Logistic Regression', '\n')
logreg_pipeline = Pipeline([('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1))])
for topic in data_df.drop(columns=['title', 'text']).columns:
    logreg_pipeline.fit(x_train, y_train[topic])
    lrp_y_pred = logreg_pipeline.predict(x_test)
    print('Test Accuracy for topic ' + topic + ': ' + str(accuracy_score(y_test[topic], lrp_y_pred)))

In [ ]:
%%time
print('Classifier Chain with Logistic Regression', '\n')
cclr_classifier = ClassifierChain(LogisticRegression())
cclr_classifier.fit(x_train, y_train)
cclr_y_pred = cclr_classifier.predict(x_test)
print('Test Accuracy: ' + str(accuracy_score(y_test, cclr_y_pred)))
print('Test Hamming Loss: ' + str(hamming_loss(y_test, cclr_y_pred)))

In [ ]:
%%time
print('Label Powerset with Logistic Regression', '\n')
lplr_classifier = LabelPowerset(LogisticRegression())
lplr_classifier.fit(x_train, y_train)
lplr_y_pred = lplr_classifier.predict(x_test)
print('Test Accuracy: ' + str(accuracy_score(y_test, lplr_y_pred)))
print('Test Hamming Loss: ' + str(hamming_loss(y_test, lplr_y_pred)))

In [ ]:
%%time
print('Binary Relevance with Logistic Regression', '\n')
brlr_classifier = BinaryRelevance(LogisticRegression())
brlr_classifier.fit(x_train, y_train)
brlr_y_pred = brlr_classifier.predict(x_test)
print('Test Accuracy: ', accuracy_score(y_test, brlr_y_pred))
print('Test Hamming loss: ', hamming_loss(y_test, brlr_y_pred))